In [8]:
import findspark
findspark.init() # this must be executed before the below import

In [9]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [17]:
import pandas as pd
import time
import rtree
from rtree import index
import numpy as np
from numpy import genfromtxt

In [60]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [78]:
# consider using a partition_rtree_index to find the corresponding partition
# this is single thread
def record_2_border(row, used_dims):
    '''
    row should be a pandas row, i.e., a point
    border is the border required in rtree index
    '''
    row_used_dims = row.iloc[:,used_dims]
    row_list = row_used_dims.values.tolist()[0]
    return tuple(row_list + row_list)
   
def route_data_2_partition(dataset, used_dims, partition_index, hdfs_path, print_execution_time=False):
    '''
    parameters:
    @dataset: should be in the form of pandas dataframe here
    @partition_index: the index of partitions
    @column_names: a full list of the column names of the table, like ['_c0','_c1','_c2']
    '''   
    pid_pdf_dict = {}
    
    start_time = time.time()
    
    for i in range(len(dataset)):
        
        record = dataset[i:i+1] # row shape = (1, n_dims)
        point_border = record_2_border(record, used_dims)
        overlap_pids = list(partition_index.intersection(point_border)) # should only contains 1
        pid = overlap_pids[0]
        
        # assign this record to the corresponding partition
        if pid in pid_pdf_dict:
            #pid_pdf_dict[pid] = pid_pdf_dict[pid].append(record) # must return, cannot replace
            pid_pdf_dict[pid] = pd.concat([pid_pdf_dict[pid], record]) # a little bit faster
        else:
            pid_pdf_dict.update({pid:record})
    
    routing_time = time.time()
    
    # persist them in HDFS
    for pid, pdf in pid_pdf_dict.items():
        partition_name = 'partition_' + str(pid)+'.parquet'
        path = hdfs_path + partition_name
        #pdf.columns = column_names
        df = sqlContext.createDataFrame(pdf)
        df.write.mode('append').parquet(path)
        
    persist_time = time.time()
    
    if print_execution_time:
        print(pid_pdf_dict)
        print('data routing time: ', routing_time-start_time)
        print('data persist time: ', persist_time-routing_time)

In [79]:
def batch_data(raw_data_path, chunk_size, used_dims, partition_index, column_names, hdfs_path):
    begin_time = time.time()
    
    chunk_count = 0
    for chunk in pd.read_csv(raw_data_path, chunksize=chunk_size):
        print('current chunk: ', chunk_count)
        chunk.columns = column_names
        route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
        chunk_count += 1
    
    finish_time = time.time()
    print('total data routing and persisting time: ', finish_time - begin_time)

In [80]:
# try multi-thread

import threading

class myThread(threading.Thread):
    def __init__(self, thread_id, name, counter, parameters):
        threading.Thread.__init__(self)
        self.thread_id = thread_id
        self.name = name
        self.counter = counter
        self.parameters = parameters
        
    def run(self):
        print('start thread: ',self.thread_id, self.name)
        chunk, used_dims, partition_index, hdfs_path = self.parameters
        route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
        print('exit thread: ',self.thread_id, self.name)

max_threads = 8

def batch_data_parallel(raw_data_path, chunk_size, used_dims, partition_index, column_names, hdfs_path):
    begin_time = time.time()
    
    count = 0
    threads = []
    for chunk in pd.read_csv(raw_data_path, chunksize=chunk_size):
        print('current chunk: ', count)
        # process this chunk of data
        tid = count % max_threads
        chunk.columns = column_names
        parameters = (chunk, used_dims, partition_index, hdfs_path)
        thread = myThread(tid, 'thread_'+str(tid)+'_'+str(count), count, parameters)
        thread.start()
        threads.append(thread)
        count += 1
        
        if tid == max_threads-1:
            for t in threads:
                t.join()
            threads = []
            
    finish_time = time.time()
    print('total data routing and persisting time: ', finish_time - begin_time)

In [81]:
def kdnode_2_border(kdnode):
    lower = [domain[0] for domain in kdnode[0]]
    upper = [domain[1] for domain in kdnode[0]]
    border = tuple(lower + upper) # non interleave
    return border

def load_partitions_from_file(path):
    '''
    the loaded stretched_kdnodes: [num_dims, l1,l2,...,ln, u1,u2,...,un, size, id, pid, left_child,id, right_child_id]
    '''
    stretched_kdnodes = genfromtxt(path, delimiter=',')
    num_dims = int(stretched_kdnodes[0,0])
    kdnodes = []
    
    for i in range(len(stretched_kdnodes)):
        domains = [ [stretched_kdnodes[i,k+1],stretched_kdnodes[i,1+num_dims+k]] for k in range(num_dims) ]
        row = [domains]
        row.append(stretched_kdnodes[i,2*num_dims+1])
        
        # to be compatible with qd-tree's partition, that do not have the last 4 attributes
        if len(stretched_kdnodes[i]) > 2*num_dims+2:
            row.append(stretched_kdnodes[i,-4])
            row.append(stretched_kdnodes[i,-3])
            row.append(stretched_kdnodes[i,-2])
            row.append(stretched_kdnodes[i,-1])
    
        kdnodes.append(row)
    
    return kdnodes

In [82]:
# path = '/home/cloudray/NORA_Partitions/nora_partitions'
path = '/home/cloudray/NORA_Partitions/qd_tree_partitions'

partitions = load_partitions_from_file(path)

for i in range(len(partitions)):
    if partitions[i][0][0][0] > partitions[i][0][0][1] or partitions[i][0][1][0] > partitions[i][0][1][1]:
        print('found invalid position: ',i, partitions[i])

p = index.Property()
p.leaf_capacity = 100 # cannot be less than 100, indicate the maximum capacity
p.fill_factor = 0.5
p.overwrite = True

partition_index = index.Index(properties = p)
for i in range(len(partitions)):
    #partition_index.insert(int(partitions[i][-4]), kdnode_2_border(partitions[i]))
    partition_index.insert(i, kdnode_2_border(partitions[i]))

In [86]:
print(len(partitions))
print(partitions[52])

53
[[[97878.09501477821, 154935.1018704741], [4167.142808958858, 6475.045713189126]], 196928.0]


In [84]:
raw_data_path = '/home/cloudray/Downloads/TPCH_12M_8Field.csv'
chunk_size = 10000
column_names = ['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7'] # handle this
hdfs_path = 'hdfs://localhost:9000/user/cloudray/NORA/'
partition_and_query_dims = [1,2]

In [ ]:
# batch_data(raw_data_path, chunk_size, partition_and_query_dims, partition_index, column_names, hdfs_path)

In [85]:
batch_data_parallel(raw_data_path, chunk_size, partition_and_query_dims, partition_index, column_names, hdfs_path)

current chunk:  0
start thread:  0 thread_0_0
current chunk:  1
start thread:  1 thread_1_1
current chunk:  2
start thread:  2 thread_2_2
current chunk:  3
start thread:  3 thread_3_3
current chunk:  4
start thread:  4 thread_4_4
current chunk:  5
start thread:  5 thread_5_5
current chunk:  6
start thread:  6 thread_6_6
current chunk:  7
start thread:  7 thread_7_7


Exception in thread thread_2_2:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-80-9b850445d3fd>", line 16, in run
    route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
  File "<ipython-input-78-95f0509d16f2>", line 28, in route_data_2_partition
    pid = overlap_pids[0]
IndexError: list index out of range

Exception in thread thread_5_5:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-80-9b850445d3fd>", line 16, in run
    route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
  File "<ipython-input-78-95f0509d16f2>", line 28, in route_data_2_partition
    pid = overlap_pids[0]
IndexError: list index out of range

Exception in thread thread_1_1:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/lib/python

Exception in thread thread_0_0:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-80-9b850445d3fd>", line 16, in run
    route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
  File "<ipython-input-78-95f0509d16f2>", line 45, in route_data_2_partition
    df.write.mode('append').parquet(path)
  File "/usr/local/spark/python/pyspark/sql/readwriter.py", line 936, in parquet
    self._jwrite.parquet(path)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1305, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 131, in deco
    return f(*a, **kw)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: An error occurred while calling

exit thread:  4 thread_4_4
current chunk:  8
start thread:  0 thread_0_8
current chunk:  9
start thread:  1 thread_1_9
current chunk:  10
start thread:  2 thread_2_10
current chunk:  11
start thread:  3 thread_3_11


Exception in thread thread_0_8:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-80-9b850445d3fd>", line 16, in run
    route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
  File "<ipython-input-78-95f0509d16f2>", line 28, in route_data_2_partition
    pid = overlap_pids[0]
IndexError: list index out of range



...

Exception in thread thread_7_1183:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-80-9b850445d3fd>", line 16, in run
    route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
  File "<ipython-input-78-95f0509d16f2>", line 28, in route_data_2_partition
    pid = overlap_pids[0]
IndexError: list index out of range

Exception in thread thread_0_1184:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-80-9b850445d3fd>", line 16, in run
    route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
  File "<ipython-input-78-95f0509d16f2>", line 28, in route_data_2_partition
    pid = overlap_pids[0]
IndexError: list index out of range

Exception in thread thread_1_1185:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/l

current chunk:  1185
start thread:  1 thread_1_1185
current chunk:  1186
start thread:  2 thread_2_1186
current chunk:  1187
start thread:  3 thread_3_1187
current chunk:  1188
start thread:  4 thread_4_1188
current chunk:  1189
start thread:  5 thread_5_1189
current chunk:  1190
start thread:  6 thread_6_1190
current chunk:  1191
start thread:  7 thread_7_1191


Exception in thread thread_5_1189:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-80-9b850445d3fd>", line 16, in run
    route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
  File "<ipython-input-78-95f0509d16f2>", line 28, in route_data_2_partition
    pid = overlap_pids[0]
IndexError: list index out of range

Exception in thread thread_6_1190:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-80-9b850445d3fd>", line 16, in run
    route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
  File "<ipython-input-78-95f0509d16f2>", line 28, in route_data_2_partition
    pid = overlap_pids[0]
IndexError: list index out of range

Exception in thread thread_7_1191:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/l

current chunk:  1192
start thread:  0 thread_0_1192
current chunk:  1193
start thread:  1 thread_1_1193
current chunk:  1194
start thread:  2 thread_2_1194
current chunk:  1195
start thread:  3 thread_3_1195
current chunk:  1196
start thread:  4 thread_4_1196
current chunk:  1197
start thread:  5 thread_5_1197
current chunk:  1198
start thread:  6 thread_6_1198
current chunk:  1199
start thread:  7 thread_7_1199
total data routing and persisting time:  175.99941635131836


Exception in thread thread_4_1196:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-80-9b850445d3fd>", line 16, in run
    route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
  File "<ipython-input-78-95f0509d16f2>", line 28, in route_data_2_partition
    pid = overlap_pids[0]
IndexError: list index out of range

Exception in thread thread_5_1197:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-80-9b850445d3fd>", line 16, in run
    route_data_2_partition(chunk, used_dims, partition_index, hdfs_path)
  File "<ipython-input-78-95f0509d16f2>", line 28, in route_data_2_partition
    pid = overlap_pids[0]
IndexError: list index out of range

Exception in thread thread_6_1198:
Traceback (most recent call last):
  File "/home/cloudray/anaconda3/l

In [87]:
df = sqlContext.read.parquet('hdfs://localhost:9000/user/cloudray/NORA/partition_19.parquet')

In [88]:
print(df)

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: double, _c5: double, _c6: double, _c7: double]


In [89]:
df.head(10)

[Row(_c0=116804.0, _c1=196223.0, _c2=11251.0, _c3=4.0, _c4=41.0, _c5=54088.02, _c6=0.0, _c7=0.08),
 Row(_c0=116868.0, _c1=200865.0, _c2=10886.0, _c3=3.0, _c4=47.0, _c5=82994.94999999998, _c6=0.08, _c7=0.06),
 Row(_c0=117158.0, _c1=197565.0, _c2=12593.0, _c3=1.0, _c4=17.0, _c5=28263.52, _c6=0.03, _c7=0.03),
 Row(_c0=117317.0, _c1=203004.0, _c2=13025.0, _c3=1.0, _c4=39.0, _c5=35372.61, _c6=0.04, _c7=0.02),
 Row(_c0=117602.0, _c1=200437.0, _c2=10458.0, _c3=3.0, _c4=44.0, _c5=58846.48, _c6=0.06, _c7=0.05),
 Row(_c0=117762.0, _c1=196124.0, _c2=11152.0, _c3=3.0, _c4=16.0, _c5=19521.92, _c6=0.05, _c7=0.05),
 Row(_c0=118086.0, _c1=200293.0, _c2=10314.0, _c3=5.0, _c4=45.0, _c5=53697.6, _c6=0.08999999999999998, _c7=0.05),
 Row(_c0=118116.0, _c1=203061.0, _c2=13082.0, _c3=1.0, _c4=50.0, _c5=48202.5, _c6=0.01, _c7=0.05),
 Row(_c0=118407.0, _c1=200424.0, _c2=10445.0, _c3=1.0, _c4=7.0, _c5=9270.87, _c6=0.06, _c7=0.06),
 Row(_c0=118471.0, _c1=195817.0, _c2=10845.0, _c3=2.0, _c4=31.0, _c5=59297.11, _c